In [1]:
from qkeras import *
from tensorflow.keras.layers import Input, AveragePooling2D, Flatten, Softmax, Add, ZeroPadding2D, MaxPooling2D
import numpy as np
from collections import namedtuple
import pickle
import math
import tensorflow as tf
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint, LearningRateScheduler
from tensorflow.keras.callbacks import ReduceLROnPlateau
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.utils import plot_model
from tensorflow.keras.utils import to_categorical
from qkeras.utils import model_save_quantized_weights

c:\ProgramData\Miniconda3\envs\qkeras\lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
c:\ProgramData\Miniconda3\envs\qkeras\lib\site-packages\numpy\.libs\libopenblas.FB5AE2TYXYH2IJRDKGDGQ3XBKLKTF43H.gfortran-win_amd64.dll
c:\ProgramData\Miniconda3\envs\qkeras\lib\site-packages\numpy\.libs\libopenblas64__v0.3.21-gcc_10_3_0.dll
  warnings.warn("loaded more than 1 DLL from .libs:"


In [2]:
def load_data(num_classes=10, subtract_pixel_mean=True):
    """
    Load CIFAR10 data and normalize
    """
    (x_train, y_train), (x_test, y_test) = cifar10.load_data()

    # input image dimensions.
    input_shape = x_train.shape[1:]

    # normalize data.
    x_train = x_train.astype('float32') / 128.0 - 1.0
    x_test = x_test.astype('float32') / 128.0 - 1.0

    # if subtract pixel mean is enabled
    if subtract_pixel_mean:
        x_train_mean = np.mean(x_train, axis=0)
        x_train -= x_train_mean
        x_test -= x_train_mean

    print('x_train shape:', x_train.shape)
    print(x_train.shape[0], 'train samples')
    print(x_test.shape[0], 'test samples')
    print('y_train shape:', y_train.shape)

    # convert class vectors to binary class matrices,
    # i.e., one hot encodings
    y_train = to_categorical(y_train, num_classes)
    y_test = to_categorical(y_test, num_classes)

    return x_train, y_train, x_test, y_test


In [3]:
x_train, y_train, x_test, y_test = load_data(10, False)

x_train shape: (50000, 32, 32, 3)
50000 train samples
10000 test samples
y_train shape: (50000, 1)


In [4]:
input_shape = x_train.shape[1:-1] + (3,)
np.random.seed(1)

a_0 = 'quantized_relu(8,0,negative_slope=0.125)'
a_1 = 'quantized_relu(8,1,negative_slope=0.125)'
a_2 = 'quantized_relu(8,2,negative_slope=0.125)'
a_3 = 'quantized_relu(8,3,negative_slope=0.125)'

q_0 = 'quantized_bits(8,0,False,True,1)'
q_1 = 'quantized_bits(8,1,False,True,1)'
q_2 = 'quantized_bits(8,2,False,True,1)'
q_3 = 'quantized_bits(8,3,False,True,1)'

q_t = 'quantized_bits(8,0,False,True,1)'

np.random.seed(42)
#preamble = './drive/MyDrive/resnet/'
preamble = ''
USE_BIAS = True

In [5]:
class Bundle(tf.keras.Model):
    def __init__(self, serial,       # int, specify the sequence of the bundle, start from 0, seek to automate!
                 core_type,          # str, Mandatory: must be conv or dense
                 core_params,        # dict, Mandaroty: parameters for conv/dense layer
                 act_core,           # str, Mandatory, can be quantization or relu

                 act_add=None,       # str, Mandatory if x1 is not None in call(), else ignored

                 pool_type=None,     # str, Optional: can only be max or avg
                 pool_params=None,  # dict, Mandatory if pool_type is not None, otherwise ignored
                 act_pool=None,      # str, Mandatory if pool_type is not None, else ignored
                 flatten=False,       # Optional: set to True to flatten the outputs
                 softmax=False,       # Optional: set to Ture to include softmax layer

                 **kwargs):

        super(Bundle, self).__init__()

        self.serial = serial
        self.core_type = core_type
        self.core_params = core_params
        self.act_core = act_core
        self.act_add = act_add
        self.pool_type = pool_type
        self.pool_params = pool_params
        self.act_pool = act_pool
        self.flatten = flatten
        self.softmax = softmax

        self.init_sanity_check()

        # Parameters derived from
        self.core_layer = None
        self.act_core_layer = None
        self.act_add_layer = None
        self.pool_layer = None
        self.act_pool_layer = None
        self.flatten_layer = None
        self.softmax_layer = None

        # Store reference to bundle object here, not just a serial number
        self.prev_bundle = None
        self.add_bundle = None

        if self.core_type == 'conv':
            self.core_layer = QConv2DBatchnorm(filters=self.core_params['filters'], kernel_size=self.core_params['kernel_size'], strides=self.core_params['strides'],
                                               padding=self.core_params['padding'], kernel_quantizer=self.core_params['kernel_quantizer'], bias_quantizer=self.core_params['bias_quantizer'], use_bias=self.core_params['use_bias'])
        else:
            self.core_layer = QDense(units=self.core_params['units'], kernel_quantizer=self.core_params['kernel_quantizer'],
                               bias_quantizer=self.core_params['bias_quantizer'], use_bias=self.core_params['use_bias'])

        self.act_core_layer = QActivation(self.act_core)

        if self.act_add is not None:
            self.act_add_layer= QActivation(self.act_add)

        if self.pool_type == 'max':
            self.pool_layer = MaxPooling2D(self.pool_params['size'], strides=self.pool_params['strides'], padding=self.pool_params['padding'])
            self.act_pool_layer = QActivation(self.act_pool)
        elif self.pool_type == 'avg':
            self.pool_layer = QAveragePooling2D(self.pool_params['size'], strides=self.pool_params['strides'], padding=self.pool_params['padding'])
            self.act_pool_layer = QActivation(self.act_pool)


        if self.flatten:
            self.flatten_layer = Flatten()

        if self.softmax:
            self.softmax_layer = Activation("softmax")

        # reference output results, from a given input
        self.y_fp = None
        self.y_int = self.y_bits = self.y_frac = None
        self.post_core = self.post_add = self.post_pool = None # fp only

        # For inference, and export only, int, bits, frac
        self.x_in = self.a_in = self.x_out = None, None, None  # For the process function!
        self.act_core_dict = self.act_add_dict = self.act_pool_dict = None
        self.w = self.b = None, None, None
        self.bundle_list = None


    def init_sanity_check(self):
        required_CONV = ['filters', 'kernel_size', 'strides', 'padding', 'kernel_quantizer', 'bias_quantizer', 'use_bias']
        required_POOL = ['size', 'strides', 'padding']
        required_DENSE = ['units', 'kernel_quantizer', 'bias_quantizer', 'use_bias']

        assert (type(self.serial) is int), f"'{type(config['serial'])} must be an integer!"

        if self.core_type == 'conv':
            for i in required_CONV:
                assert i in self.core_params, f"'{i}' must be provided for conv"
        elif self.core_type == 'dense':
            for i in required_DENSE:
                assert i in self.core_params, f"'{i}' must be provided for dense"
        else:
            raise Exception(self.core_type, "a core type must be provided, only conv or dense supported for now")

        assert self.act_core is not None, "Must provide activation for core"

        if self.pool_type is not None:
            if self.pool_type not in ['avg', 'max']:
                raise Exception(self.pool_type, "only avg or max supported for now")
            for i in required_POOL:
                assert i in self.pool_params, f"'{i}' must be provided for pooling"

            assert self.act_pool is not None, "Must provide activation post pooling"

        return

    # functions for training
    def call(self, x, x_1=None):
        if hasattr(x, "bundleObj"):
            self.prev_bundle = x.bundleObj
        else:
            self.prev_bundle = -1
            # -1 indicates input layer, which do not belong to any bundle

        x = self.core_layer(x)
        x = self.act_core_layer(x)
        if x_1 is not None:
            if hasattr(x_1, "bundleObj"):
                self.add_bundle = x_1.bundleObj
            else:
                self.add_bundle = -1
                # -1 indicates input layer, which do not belong to any bundle
            x = Add()([x, x_1])
            x = self.act_add_layer(x)
        if self.pool_layer:
            x = self.pool_layer(x)
            x = self.act_pool_layer(x)
        if self.flatten_layer:
            x = self.flatten_layer(x)
        if self.softmax_layer:
            x = self.softmax_layer(x)

        x.bundleObj = self

        return x

    # functions to be prepared for exportation
    def load_weight_bias(self):
        #print('check serial', self.serial)
        k = self.core_layer.get_folded_weights()[0] if isinstance(self.core_layer, QConv2DBatchnorm) else self.core_layer.kernel
        k = self.core_layer.kernel_quantizer_internal(k).numpy()
        k_config = self.core_layer.kernel_quantizer_internal.get_config()

        k_frac = k_config['bits']-k_config['integer']-k_config['keep_negative']
        k_int = k * 2**k_frac
        assert (k_int == k_int.astype(int)).all()
        k_int = k_int.astype(int)

        self.w = k_int, k_config['bits'], k_frac

        if (self.core_type == 'conv' and self.core_params['use_bias']) or (self.core_type == 'dense' and self.core_params['use_bias']):
            b = self.core_layer.get_folded_weights()[1] if isinstance(self.core_layer, QConv2DBatchnorm) else self.core_layer.bias
            b = self.core_layer.bias_quantizer_internal(b).numpy()
            b_config = self.core_layer.bias_quantizer_internal.get_config()
            b_frac = b_config['bits']-b_config['integer']-b_config['keep_negative']
            b_int = b * 2**b_frac
            assert (b_int == b_int.astype(int)).all()
            b_int = b_int.astype(int)
            self.b = b_int, b_config['bits'], b_frac

    def prepare_val(self, y_fp, x_init=None):
        self.load_weight_bias()
        self.y_fp = y_fp

        # input tensor
        x_ref = None
        if self.serial == 0:
            x_ref = x_init
        else:
            x_ref = self.prev_bundle.y_fp

        x = self.core_layer(x_ref)
        x = self.act_core_layer(x)
        self.post_core = x.numpy()

        if self.add_bundle is not None:
            x1 = self.add_bundle.y_fp
            x = Add()([x, x1])
            self.post_add = x.numpy()
            x = self.act_add_layer(x)
            self.post_add_act = x.numpy()
        if self.pool_layer:
            x = self.pool_layer(x)
            x = self.act_pool_layer(x)
            self.post_pool = x.numpy()
        if self.flatten_layer:
            x = self.flatten_layer(x)
        if self.softmax_layer:
            x = self.softmax_layer(x)

        def extract_act(ilayer):
            d = ilayer.quantizer.get_config()
            sign_bit = d['keep_negative'] if 'keep_negative' in d else (d['negative_slope'] !=0 if 'negative_slope' in d else (0))
            int_bit = d['integer'] if 'integer' in d else 0
            frac = d['bits']-int_bit-sign_bit
            if isinstance(ilayer.quantizer, quantized_bits):
                return {'type':'quant', 'bits':d['bits'], 'frac':frac}
            elif 'relu' in str(ilayer.quantizer.__class__):
                return {'type':'relu', 'slope':ilayer.quantizer.negative_slope, 'bits':d['bits'], 'frac':frac}
            else:
                raise Exeption("Only relu is suppported!")

        self.act_core_dict = extract_act(self.act_core_layer)
        self.y_frac, self.y_bits = self.act_core_dict['frac'], self.act_core_dict['bits']

        if self.add_bundle is not None:
            self.act_add_dict = extract_act(self.act_add_layer)
            self.y_frac, self.y_bits = self.act_add_dict['frac'], self.act_add_dict['bits']

        if self.pool_layer:
            self.act_pool_dict = extract_act(self.act_pool_layer)
            self.y_frac, self.y_bits = self.act_pool_dict['frac'], self.act_pool_dict['bits']

        if self.softmax_layer:
            self.y_frac, self.y_bits = 0, 1

        self.y_int = self.y_fp * 2**self.y_frac

        # no point of assertion in case of softmax, exponential used!
        if self.softmax is None:
            assert (self.y_int == self.y_int.astype(int)).all(), self.serial


    def process_val(self, function, x = None):
        # x is none: chained mode, read x from prev layer y
        # x is not none: independent mode, inputs fed by outputs. Must provide input for the first bundle
        if x is not None:
            self.x_in = x
        else:
            # ToDo: do not rely on external(global) variables!
            prev_bdl = self.prev_bundle
            self.x_in = prev_bdl.y_int, prev_bdl.y_bits, prev_bdl.y_frac
            assert self.serial > 0, "input must be provided manually for the first bundle"

        def quantize(x, bits, frac):
            x = x.astype(np.float32)
            x /= 2 ** frac
            x = np.around(x)
            x = np.clip(x, -2**(bits-1), 2**(bits-1)-1)
            x = x.astype(int)
            return x

        x_arr, x_bits, x_frac = self.x_in
        x_fp = x_arr / (2**x_frac)

        w_arr, w_bits, w_frac = self.w

        out_bits, out_frac = x_bits + w_bits, x_frac + w_frac
        # ToDo: fix the out bits, addition not considered!
        out_arr = function(x_arr, w_arr)

        if self.b[0] is not None:
            b_arr, b_bits, b_frac = self.b
            out_arr += b_arr * 2** (out_frac - b_frac)

        if 'strides' in self.core_params and self.core_params['strides'] != (1,1):
            SH, SW = self.core_params['strides']
            N, XH, XW, C = out_arr.shape
            YH, YW = XH//SH, XW//SW
            out_arr = out_arr.reshape(N, YH, SH, YW, SW, C)
            ind = -1 if w_arr.shape[0] > 1 else 0
            out_arr = out_arr[:,:,ind,:,ind,:]

        def process_act(in_arr, in_bits, in_frac, act_dict):

            if act_dict['type'] == 'quant':
                out_arr = quantize(x=in_arr, bits=act_dict['bits'], frac=in_frac-act_dict['frac'])
                out_frac = act_dict['frac']
                out_bits = act_dict['bits']

            elif act_dict['type'] == 'relu':
                frac, bits = act_dict['frac'], act_dict['bits']

                out_arr = in_arr * 2**(frac-in_frac)
                out_arr = np.clip(out_arr, -2**(bits-1), 2**(bits-1)-1)

                out_arr = np.maximum(out_arr * act_dict['slope'], out_arr)
                out_arr = np.around(out_arr)
                out_arr = np.clip(out_arr,-2**(bits-1), 2**(bits-1)-1).astype(int)

                out_frac, out_bits = frac, bits
            else:
                raise Exception('Only relu is supported yet')

            return out_arr, out_bits, out_frac

        out_arr, out_bits, out_frac = process_act(out_arr, out_bits, out_frac, self.act_core_dict)
        assert np.all(out_arr == self.post_core * 2**out_frac)

        if self.add_bundle is not None:
            part_bdl = self.add_bundle

            added_fp = self.post_core+part_bdl.y_fp

            a_arr, a_bits, a_frac = part_bdl.y_int, part_bdl.y_bits, part_bdl.y_frac
            out_frac_add, out_bits_add = max(out_frac, a_frac), max(out_bits, a_bits)
            a_arr_cast = a_arr * 2** (out_frac_add - a_frac)
            out_arr_cast = out_arr * 2 **(out_frac_add - out_frac)
            out_arr = out_arr_cast.astype(np.int64) + a_arr_cast.astype(np.int64)

            out_bits, out_frac = out_bits_add, out_frac_add

            assert np.all(out_arr == self.post_add * 2**out_frac)
            #print(out_bits, out_frac)
            #print(out_arr[0,:,:,0])
            if self.act_add:
                #out_arr = added_fp * 2**out_frac
                out_arr, out_bits, out_frac = process_act(out_arr, out_bits, out_frac, self.act_add_dict)

            #print(self.act_add_dict, out_frac)
            #print(added_fp[0,:,:,0])
            #print(out_arr[0,:,:,0])
            #print(self.act_add_dict)

            #print(out_frac, self.post_add_act[0,:,:,0]*2**out_frac)
            #print(self.post_add[0,:,:,0] * 2**out_frac)
            assert np.all(out_arr == self.post_add_act * 2**out_frac)

        if self.pool_layer:
            if self.pool_type == 'max':
                pStride = self.pool_params['strides']
                pSize = self.pool_params['size']

                def findMax(InArray, p, q):
                    results = np.zeros((InArray.shape[0], InArray.shape[3]))
                    results -= math.inf
                    for i in range(p, p+pSize[0]):
                        for j in range(q, q+pSize[1]):
                            if i >=0 and j>=0 and i < InArray.shape[1] and j < InArray.shape[2]:
                                cand = InArray[:,i,j,:]
                                results = np.maximum(results, cand)
                    return results

                def HotFixMaxPool2D(InArray):
                    if pStride[0]!=pStride[1] or pSize[0]!=pSize[1]:
                        raise Exception('Only square stride and size is supported')
                    if pSize[0]/2 == 0:
                        raise Exception('Maxpool size should be odd')

                    pad = (pSize[0]-1)//2

                    inShape = InArray.shape
                    assert len(inShape) == 4
                    OutArray = np.zeros((inShape[0], inShape[1]//pStride[0], inShape[2]//pStride[1], inShape[3]))
                    # Start point, should include pad
                    st_p, st_q = -pad, -pad

                    for i in range(OutArray.shape[1]):
                        for j in range(OutArray.shape[2]):
                            p, q = st_p + i*pStride[0] + pStride[0]-1, st_q + j*pStride[1] + pStride[1]-1
                            OutArray[:,i,j,:] = findMax(InArray, p, q)

                    return OutArray

                out_arr = HotFixMaxPool2D(out_arr).astype(int)

            elif self.pool_type == 'avg':
                assert self.pool_params['size'] == self.pool_params['strides']
                KH, KW = self.pool_params['size']
                N, H, W, C = out_arr.shape
                out_arr = out_arr.reshape(N, H//KH, KH, W//KW, KW, C).mean(axis=(2,4))
                # NO need for clipping, as act_pool in place!

            if self.act_pool:
                out_arr, out_bits, out_frac = process_act(out_arr, out_bits, out_frac, self.act_pool_dict)
            assert np.all(out_arr == self.post_pool * 2**out_frac)

        if self.flatten:
            out_arr = out_arr.reshape(out_arr.shape[0],-1)



        if self.softmax:
            out_arr = out_arr / 2**out_frac
            exp = np.exp(out_arr - out_arr.max())
            out_arr = exp/np.sum(exp, axis=1)[0]
            assert np.all(np.argmax(self.y_int, axis=-1) == np.argmax(out_arr, axis=-1))
        else:
            assert np.all(out_arr == self.y_int)


        return





In [6]:
x = x_in = Input(input_shape, name='input')
x = QActivation(q_0)(x)

x = x1 = Bundle(0, 'conv', {'filters':64, 'kernel_size':(7,7), 'strides':(2,2), 'padding':'same', 'kernel_quantizer':q_0, 'bias_quantizer':q_0, 'use_bias':USE_BIAS}, a_0,
          None, 'max', {'size':(3,3), 'strides':(1,1), 'padding':'same'}, q_0)(x)
# block 0
x = Bundle(1, 'conv', {'filters':64, 'kernel_size':(3,3), 'strides':(1,1), 'padding':'same', 'kernel_quantizer':q_0, 'bias_quantizer':q_0, 'use_bias':USE_BIAS}, a_0)(x)
x = x1 = Bundle(2, 'conv', {'filters':64, 'kernel_size':(3,3), 'strides':(1,1), 'padding':'same', 'kernel_quantizer':q_0, 'bias_quantizer':q_0, 'use_bias':USE_BIAS},
                 q_1, a_0)(x, x1)

x = Bundle(3, 'conv', {'filters':64, 'kernel_size':(3,3), 'strides':(1,1), 'padding':'same', 'kernel_quantizer':q_0, 'bias_quantizer':q_0, 'use_bias':USE_BIAS}, a_0)(x)
x = x1 = Bundle(4, 'conv', {'filters':64, 'kernel_size':(3,3), 'strides':(1,1), 'padding':'same', 'kernel_quantizer':q_0, 'bias_quantizer':q_0, 'use_bias':USE_BIAS},
                 q_1, a_1)(x, x1)
# block 1
x1 = Bundle(5, 'conv', {'filters':128, 'kernel_size':(3,3), 'strides':(2,2), 'padding':'same', 'kernel_quantizer':q_0, 'bias_quantizer':q_0, 'use_bias':USE_BIAS}, a_1)(x1)
x1 = Bundle(6, 'conv', {'filters':128, 'kernel_size':(3,3), 'strides':(1,1), 'padding':'same', 'kernel_quantizer':q_0, 'bias_quantizer':q_0, 'use_bias':USE_BIAS}, q_2)(x1)
x = x1 = Bundle(7, 'conv', {'filters':128, 'kernel_size':(1,1), 'strides':(2,2), 'padding':'same', 'kernel_quantizer':q_0, 'bias_quantizer':q_0, 'use_bias':USE_BIAS},
                 q_2, a_2)(x, x1)

x = Bundle(8, 'conv', {'filters':128, 'kernel_size':(3,3), 'strides':(1,1), 'padding':'same', 'kernel_quantizer':q_0, 'bias_quantizer':q_0, 'use_bias':USE_BIAS}, a_1)(x)
x = x1 = Bundle(9, 'conv', {'filters':128, 'kernel_size':(1,1), 'strides':(1,1), 'padding':'same', 'kernel_quantizer':q_0, 'bias_quantizer':q_0, 'use_bias':USE_BIAS},
                 q_2, a_2)(x, x1)

#block 2
x1 = Bundle(10, 'conv', {'filters':256, 'kernel_size':(3,3), 'strides':(2,2), 'padding':'same', 'kernel_quantizer':q_0, 'bias_quantizer':q_0, 'use_bias':USE_BIAS}, a_1)(x1)
x1 = Bundle(11, 'conv', {'filters':256, 'kernel_size':(3,3), 'strides':(1,1), 'padding':'same', 'kernel_quantizer':q_0, 'bias_quantizer':q_0, 'use_bias':USE_BIAS}, q_2)(x1)
x = x1 = Bundle(12, 'conv', {'filters':256, 'kernel_size':(1,1), 'strides':(2,2), 'padding':'same', 'kernel_quantizer':q_0, 'bias_quantizer':q_0, 'use_bias':USE_BIAS},
                 q_3, a_3)(x, x1)

x = Bundle(13, 'conv', {'filters':256, 'kernel_size':(3,3), 'strides':(1,1), 'padding':'same', 'kernel_quantizer':q_0, 'bias_quantizer':q_0, 'use_bias':USE_BIAS}, a_2)(x)
x = x1 = Bundle(14, 'conv', {'filters':256, 'kernel_size':(1,1), 'strides':(1,1), 'padding':'same', 'kernel_quantizer':q_0, 'bias_quantizer':q_0, 'use_bias':USE_BIAS},
                 q_3, a_3)(x, x1)

#block 3
x1 = Bundle(15, 'conv', {'filters':512, 'kernel_size':(3,3), 'strides':(2,2), 'padding':'same', 'kernel_quantizer':q_0, 'bias_quantizer':q_0, 'use_bias':USE_BIAS}, a_2)(x1)
x1 = Bundle(16, 'conv', {'filters':512, 'kernel_size':(3,3), 'strides':(1,1), 'padding':'same', 'kernel_quantizer':q_0, 'bias_quantizer':q_0, 'use_bias':USE_BIAS}, q_3)(x1)
x = x1 = Bundle(17, 'conv', {'filters':512, 'kernel_size':(1,1), 'strides':(2,2), 'padding':'same', 'kernel_quantizer':q_0, 'bias_quantizer':q_0, 'use_bias':USE_BIAS},
                 q_3, a_3)(x, x1)

x = Bundle(18, 'conv', {'filters':512, 'kernel_size':(3,3), 'strides':(1,1), 'padding':'same', 'kernel_quantizer':q_0, 'bias_quantizer':q_0, 'use_bias':USE_BIAS}, a_2)(x)
x = Bundle(19, 'conv', {'filters':512, 'kernel_size':(1,1), 'strides':(1,1), 'padding':'same', 'kernel_quantizer':q_0, 'bias_quantizer':q_0, 'use_bias':USE_BIAS},
                 q_3, a_3, 'avg', {'size':(2,2), 'strides':(2,2), 'padding':'valid'}, q_3, True)(x, x1)

x = Bundle(20, 'dense', {'units':10, 'kernel_quantizer':q_2, 'bias_quantizer':q_2, 'use_bias':USE_BIAS}, q_3, softmax=True)(x)

model = Model(inputs=x_in, outputs=x)
print(model.summary(expand_nested=True))

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input (InputLayer)             [(None, 32, 32, 3)]  0           []                               
                                                                                                  
 q_activation (QActivation)     (None, 32, 32, 3)    0           ['input[0][0]']                  
                                                                                                  
 bundle (Bundle)                (None, 16, 16, 64)   9729        ['q_activation[0][0]']           
|¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯|
| q_conv2d_batchnorm (QConv2DBat  multiple          9729        []                               |
| chnorm)                                                                                     

In [7]:
def lr_schedule(epoch):
    """
  Bundles_pre_trainearning Rate Schedule
    Learning rate is scheduled to be reduced after 80, 120, 160, 180 epochs.
    Called automatically every epoch as part of callbacks during training.
    # Arguments
        epoch (int): The number of epochs
    # Returns
        lr (float32): learning rate
    """
    # initial_lr = 1e-4
    # lr_decay = 0.99
    # lr = initial_lr * (lr_decay ** epoch)
    lr = 1e-3 # default 1e-3
    if epoch > 180:
        lr *= 0.5e-3
    elif epoch > 150:
        lr *= 1e-2
    elif epoch > 100:
        lr *= 1e-1
    elif epoch > 50:
        lr *= 1e-1
    print('Learning rate: ', lr)
    return lr

preamble = ''
model_file_path = preamble+'resnet18.h5'
checkpoint = ModelCheckpoint(filepath=model_file_path,
                                     monitor='val_acc',
                                     verbose=1,
                                     save_best_only=True)
lr_reducer = ReduceLROnPlateau(factor=np.sqrt(0.1),
                                       cooldown=0,
                                       patience=5,
                                       min_lr=0.5e-6)
lr_scheduler = LearningRateScheduler(lr_schedule)

callbacks = [checkpoint, lr_reducer, lr_scheduler]

NB_EPOCH = 200
BATCH_SIZE = 256
VERBOSE = 1
VALIDATION_SPLIT = 0.1
RELU_NEG_SLOPE = 0.125

model.compile(loss='categorical_crossentropy',
            optimizer=Adam(learning_rate=lr_schedule(0)), metrics=['acc'])

# model.fit(x_train, y_train,
#                     batch_size=BATCH_SIZE,
#                     epochs=NB_EPOCH,
#                     validation_data=(x_test, y_test),
#                     shuffle=True,
#                     callbacks=callbacks)

Learning rate:  0.001


In [8]:
#arch = model.to_json()
#with open("bundle_resnet_18.json", 'w') as arch_file:
#    arch_file.write(arch)

In [9]:
def conv(x,w):
    x = x.astype(np.int32)
    w = w.astype(np.int32)
    return tf.keras.backend.conv2d(x, w, padding='same').numpy()

In [10]:
XN = 4
x = np.random.randn(XN, *model.input.shape[1:])
x = np.clip(x, -1.0, 1.0)
print(x.shape)

pre_layer = model.layers[1]
temp_model = Model(inputs=model.input, outputs=pre_layer.output)
x_init = temp_model(x, training=False)

for i, layer in enumerate(model.layers[2:]): # disregard input and initial quant layers
    print(layer.name)
    temp_model = Model(inputs=model.input, outputs=layer.output)
    y = temp_model(x, training=False).numpy()
    x_init = x_init if i == 0 else None
    layer.prepare_val(y, x_init)

(4, 32, 32, 3)
bundle


bundle_1
bundle_2
bundle_3
bundle_4
bundle_5
bundle_6
bundle_7
bundle_8
bundle_9
bundle_10
bundle_11
bundle_12
bundle_13
bundle_14
bundle_15
bundle_16
bundle_17
bundle_18
bundle_19
bundle_20


In [11]:
init_layer = model.layers[1]
temp_model = Model(inputs=model.input, outputs=init_layer.output)
x_init = temp_model(x, training=False).numpy()
x_init_bits = 8
x_init_frac = 7
x_init_arr = x_init * 2**x_init_frac

model.layers[2].process_val(conv, (x_init_arr, x_init_bits, x_init_frac))
for j in range(3, len(model.layers)-1):
    model.layers[j].process_val(conv)

model.layers[-1].process_val((lambda x, w : x @ w))

In [14]:
model.layers[2].y_int

array([[[[122.,  72.,  30., ..., 127., 127., 100.],
         [122.,  72.,  84., ..., 127., 127., 100.],
         [127., 127.,  84., ..., 100., 127.,  91.],
         ...,
         [116.,  77., 127., ..., 127., 127., 110.],
         [116., 127., 127., ..., 127., 119., 127.],
         [116., 127., 127., ..., 127., 119., 127.]],

        [[127.,  72.,  30., ..., 127., 127., 127.],
         [127.,  72.,  84., ..., 127., 127., 127.],
         [127., 127.,  84., ..., 127., 127., 127.],
         ...,
         [116.,  77., 127., ..., 127., 127., 110.],
         [116., 127., 127., ..., 127., 119., 127.],
         [116., 127., 127., ..., 127., 119., 127.]],

        [[127.,  -1., 127., ..., 127., 127., 127.],
         [127.,   0., 127., ..., 127., 127., 127.],
         [ 72.,  55., 127., ..., 127., 127., 127.],
         ...,
         [127., 126., 127., ..., 127.,  85., 110.],
         [127., 126., 127., ..., 127., 119., 127.],
         [127.,  91., 127., ..., 127., 119., 127.]],

        ...,

  

In [15]:
model.layers[2].o_arr

AttributeError: 'Bundle' object has no attribute 'o_arr'